In [1]:
#Bootstrap 
import sys 
import os


import sklearn
assert sklearn.__version__ >= '0.20'
import numpy as np
import pandas as pd



In [2]:
from os import listdir
from os.path import isfile, join


def read_data_csv(path=None,file_names=None, particular_word=None):
    li = []

    
    for file in file_names:
        if particular_word in file:
            print(file)
        #something isnt appending to files correctly
            file = join(path, file)
            df = pd.read_csv(file, index_col=None, header=0)
       
            li.append(df)
    aggeregated_data = pd.concat(li, axis=0)
    return aggeregated_data


        

In [3]:

#load data (#combine the two csv files into one data frame)


# try:
#     mac_path = '/Users/tlesick/Dropbox/data_sources/southern_ct_shoreline/'
# #     data = #read_data_csv(mac_path, )
    
# except:
# ml_server_path = '/home/betty/Dropbox/data_sources/southern_ct_shoreline/'
# combine_files = [f for f in listdir(ml_server_path) if isfile(join(ml_server_path, f))]

# data = read_data_csv(path=ml_server_path, file_names=combine_files, particular_word='county')


    

def read_directory(path=None):
    path = '/home/betty/Dropbox/data_sources/southern_ct_shoreline/'
    files = combine_files = [f for f in listdir(ml_server_path) if isfile(join(ml_server_path, f))]
    return files


In [4]:
#load data (#combine the two csv files into one data frame)


# try:
#     mac_path = '/Users/tlesick/Dropbox/data_sources/southern_ct_shoreline/'
# #     data = #read_data_csv(mac_path, )
    
# except:
ml_server_path = '/home/betty/Dropbox/data_sources/southern_ct_shoreline/'
combine_files = [f for f in listdir(ml_server_path) if isfile(join(ml_server_path, f))]

data = read_data_csv(path=ml_server_path, file_names=combine_files, particular_word='county')


    

fairfield_county_redfin_2020-03-19-10-18-28.csv
litchfield_county_redfin_2020-03-19-10-10-25.csv
new_haven_county_redfin_2020-03-19-10-15-39.csv
windham_county_redfin_2020-03-19-10-14-06.csv
new_london_county_redfin_2020-03-19-10-04-57.csv
tolland_county_redfin_2020-03-19-10-17-44.csv
middlesex_county_redfin_2020-03-19-10-15-11.csv
hartford_county_redfin_2020-03-19-10-13-19.csv


In [5]:
# data.count()

In [6]:
# data.info()

In [7]:
# data.describe()

In [8]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# data.hist(bins=50, figsize=(20,15))
# plt.show()

In [9]:
np.random.seed(42)

In [10]:
try:
    revised_data  = data.drop( ["SOLD DATE", "ADDRESS","SALE TYPE", "CITY", "ZIP OR POSTAL CODE",
            "URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)",
            "STATUS", "SOURCE", "MLS#", "FAVORITE","NEXT OPEN HOUSE START TIME", "NEXT OPEN HOUSE END TIME", 
          "INTERESTED", "LOCATION", "HOA/MONTH", "STATE OR PROVINCE", "$/SQUARE FEET",
                ], axis=1)
except: 
    pass

#drop the rows that contain null values in city
# data = data.dropna(subset=["CITY"])
#cannot stratify if has only one town
# data = data[data.groupby('CITY').CITY.transform(len) > 2]
# data = data.rename(columns={"ZIP OR POSTAL CODE": "ZIP"})

# data = data[data.groupby('ZIP').ZIP.transform(len) > 2]

In [11]:
# data['ZIP'].value_counts().min()
# data_prep = data[data.groupby('ZIP').ZIP.tranform(len) > 1]

In [12]:
#create index
data_with_id = revised_data.reset_index()


In [13]:
#it is possible to stratify here 
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(revised_data,
                                       test_size=0.3, 
                                       random_state=42,
                                       stratify=data["PROPERTY TYPE"]
                                      )


In [38]:
#drop price from training 
x_train = train_set.drop(["PRICE",
                ], axis=1)
y_train = train_set["PRICE"].copy()

x_test = test_set.drop(["PRICE",
                ], axis=1)
y_test = test_set["PRICE"].copy()

#median numbers to fill later
housing_num_median = x_train.median().values

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer



#all columns other than property category
numerical_attributes = list(x_train.drop(["PROPERTY TYPE"], axis=1))
#only property type 
categorical_attributes = list(x_train[["PROPERTY TYPE"]])





num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])



categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(sparse=False)),
])

In [16]:
#another option is to prep the data into a simple pipeline, this example utilizes a seperate numerical and categorical
from sklearn.compose import ColumnTransformer



full_pipeline = ColumnTransformer([
    ("numerical_attributes", num_pipeline, numerical_attributes),
    ("categorical_attributes", categorical_pipeline, categorical_attributes),
 
])

x_train_prepared = full_pipeline.fit_transform(x_train)
x_test_prepared = full_pipeline.fit_transform(x_test)

In [17]:
#one more scaling feature is to combine pipelines with
#from sklearn.pipeline import FeatureUnion
#setup the same way as pipeline or columntransfer


# to see how it is transformed
# x_train_transformed = pd.DataFrame(x_train_prepared, columns=x_train.columns,
#                           index=x_train.index)

In [44]:
#try training now
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x_train_prepared, y_train)
#score is the r2
lin_reg.score(x_train_prepared, y_train)


0.48147984924414716

In [45]:
#lets see how it preformed 
some_data = x_test.iloc[:5]

some_labels = y_test.iloc[:5]


some_data_prepd = full_pipeline.transform(some_data)
# some_data_prepd.shape
lin_reg.predict(some_data_prepd)

print('Predictions: ', lin_reg.predict(some_data_prepd))
print('actual:', list(some_labels))


Predictions:  [ 117248. 1074176.  488960.  312576.  304384.]
actual: [130000, 460000, 529000, 439900, 357000]


In [26]:
#check the mean square error for performance of the linear regression
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(x_train_prepared)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

308766.4204541251

In [42]:

def compare_scores(model, x, y):
    print('Predictions: ', model.predict(x))
    print('actual:', list(y))


In [46]:
#using a decision tree model rather than the linear regression
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(x_train_prepared, y_train)
tree_reg.score(x_test_prepared, y_test)

0.31655406487257765

In [47]:
compare_scores(tree_reg, some_data_prepd, some_labels)


Predictions:  [ 200000. 1597000.  339000.  415000.  299900.]
actual: [130000, 460000, 529000, 439900, 357000]


In [49]:
#fine tune the model by splitting the training/test data into representative groups
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
tree_revised_rmnse_scores = np.sqrt(-scores)

In [50]:
lin_scores = cross_val_score(lin_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)


In [53]:
decision_tree_predictions = tree_reg.predict(x_train_prepared)
tree_mse = mean_squared_error(y_train, decision_tree_predictions)
tree_root_mse = np.sqrt(tree_mse)
tree_root_mse

73.77111135633174

In [55]:
#uses cross validation scores 
# E.G.cross_val_score(tree_reg, x_train_prepared, y_train, scoring="neg_mean_squared_error", cv=10)
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard_deviation: ", scores.std())
display_scores(tree_revised_rmnse_scores)

Scores:  [275836.58421116 456487.2401824  361944.90695936 398616.52273133
 512661.62662429 299110.43076207 385997.36033947 378613.84796773
 392379.13983692 242863.47761605]
Mean:  370451.11372307816
Standard_deviation:  77201.5882769269


In [57]:
#the linear regression actually preforms better on this smaller training set
linear_regression_scores = cross_val_score(lin_reg, x_train_prepared, y_train, scoring='neg_mean_squared_error', cv=10)
linear_regression_rmse_scores = np.sqrt(-linear_regression_scores)
display_scores(linear_regression_rmse_scores)

Scores:  [276438.70981403 400042.80879603 318104.04595663 370219.99035773
 336868.52321622 494726.93796334 315344.27804872 316292.9925013
 245622.32301833 179071.03724184]
Mean:  325273.1646914162
Standard_deviation:  81727.89952737435


In [60]:
pd.Series(np.sqrt(-scores)).describe()


count        10.000000
mean     370451.113723
std       81377.619313
min      242863.477616
25%      314819.049811
50%      382305.604154
75%      397057.177008
max      512661.626624
dtype: float64

In [62]:
#SVR model
from sklearn.svm import SVR 

#i should make a function to call each modules easily with the fit, etc. like below
svm_regression = SVR(kernel='linear')
svm_regression.fit(x_train_prepared, y_train)
housing_predictions = svm_regression.predict(x_train_prepared)
svm_mse = mean_squared_error(y_train, housing_predictions)
#unsure why sometimes it was run 
svm_rmse = np.sqrt(svm_mse)
svm_rmse

437589.38641534024

In [64]:
compare_scores(svm_regression, some_data_prepd, some_labels)

Predictions:  [270209.41710451 281149.14411321 272465.4808719  273075.45629781
 271705.83226637]
actual: [130000, 460000, 529000, 439900, 357000]


In [65]:
#check multiple types of models at once 

from sklearn.model_selection import GridSearchCV


#the first attempt will be 12 different combinations (3 estimators x 4 features)
#the second attempt will do 2 estimators by 3 features so 6 combinations
param_grid = [
    #estimators are the number of 'trees', 'features' are the parameters
    {'n_estimators':[3,10,3], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3, 10], 'max_features': [2,3,4]},
    
]

In [67]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(x_train_prepared, y_train)
forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(x_train_prepared, y_train)



GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{'max_features': [2, 4, 6, 8],
  

In [68]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [69]:
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(np.sqrt(-mean_score), params)

307112.586932478 {'max_features': 2, 'n_estimators': 3}
292026.5392362805 {'max_features': 2, 'n_estimators': 10}
307112.586932478 {'max_features': 2, 'n_estimators': 3}
307926.77870058286 {'max_features': 4, 'n_estimators': 3}
282627.49475593324 {'max_features': 4, 'n_estimators': 10}
307926.77870058286 {'max_features': 4, 'n_estimators': 3}
295282.65456900315 {'max_features': 6, 'n_estimators': 3}
270590.9173126652 {'max_features': 6, 'n_estimators': 10}
295282.65456900315 {'max_features': 6, 'n_estimators': 3}
309052.2963286775 {'max_features': 8, 'n_estimators': 3}
279219.42669122014 {'max_features': 8, 'n_estimators': 10}
309052.2963286775 {'max_features': 8, 'n_estimators': 3}
305498.13064252655 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
280320.0515603753 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
336147.00787700835 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
282319.10761667724 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [71]:
#when looking for more combinations try randomcvsearch
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_regression = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(forest_regression, param_distributions=param_distribs,
                                  n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(x_train_prepared, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [72]:
feature_importances = grid_search.best_estimator_.feature_importances_


In [73]:
# displaying the importance scores next to their columns
category_encoder = full_pipeline.named_transformers_["categorical_attributes"]["one_hot"]

cat_one_hot_attributes = list(category_encoder.categories_[0])

attributes = numerical_attributes + cat_one_hot_attributes
sorted(zip(feature_importances, attributes), reverse=True)

[(0.2335888550128886, 'BATHS'),
 (0.2026371410670815, 'SQUARE FEET'),
 (0.14786019945265713, 'LONGITUDE'),
 (0.14069813479748922, 'LATITUDE'),
 (0.10598364019902105, 'YEAR BUILT'),
 (0.09258329844156318, 'LOT SIZE'),
 (0.032115936662370385, 'BEDS'),
 (0.02613057148120026, 'DAYS ON MARKET'),
 (0.008851798222599504, 'Single Family Residential'),
 (0.003343153824670676, 'Vacant Land'),
 (0.0026977342043506794, 'Condo/Co-op'),
 (0.0021091384257115493, 'Multi-Family (2-4 Unit)'),
 (0.0013763569714035993, 'Multi-Family (5+ Unit)'),
 (2.404123699264871e-05, 'Townhouse')]

In [74]:
#I should probably remove the $/square feet, as there probably is fairly high correlation between the two
#also latitude seems like an odd predictor of price, as all data is within the town..
category_encoder.categories_


[array(['Condo/Co-op', 'Multi-Family (2-4 Unit)', 'Multi-Family (5+ Unit)',
        'Single Family Residential', 'Townhouse', 'Vacant Land'],
       dtype=object)]

In [75]:
final_model = grid_search.best_estimator_ 
final_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [ ]:
# x_test["CITY"].value_counts()

In [ ]:
# x_train = 177, x_test = 159
# x_test["ZIP"].value_counts()

In [ ]:
# x_train["ZIP"].value_counts()

In [77]:
#evaluate the model on the test set




# x_test_prepared = full_pipeline.transform(x_test)


final_predictions = final_model.predict(x_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [78]:
final_rmse

469103.1013669399

In [79]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

/home/betty/ml_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


array([            nan, 681705.03388118])